In [4]:
import requests
import pandas as pd
import json
import time
import csv
from datetime import date

name = 'vacancies'
experience = 'between1And3'  # "between1And3" or "between3And6" or "moreThan6"
# area = 113  # Россия = 113, Москва = 1
schedule = 'remote'
q = 'NAME:("аналитик" or "data аналитик" or "аналитик данных" or "BI" or "BI разработчик" or "data analyse" or "analyst")' 
today = date.today()

def get_page(page = 0):

    params = {
        'text': q, 
        # 'area': area,
        'experience': experience, 
        # 'only_with_salary': 'true',
        'schedule': schedule,
        'order_by': 'relevance',
        'page': page, 
        'per_page': 100,
    }

    req = requests.get('https://api.hh.ru/vacancies', params) 
    data = req.content.decode() # Декодируем его ответ, чтобы кириллица отображалась корректно
    req.close()
    return data

# Создаем список, в котором будут хранится ответы запроса к сервису API hh.ru
js_objs = []

# Считываем первые 2000 вакансий
for page in range(0, 20):

    # Преобразуем текст ответа запроса в словарь Python
    js_obj = json.loads(get_page(page))

    # Добавляем текущий ответ запроса в список
    js_objs.extend(js_obj['items'])

    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (js_obj['pages'] - page) <= 1:
        break

    # Необязательная задержка, но чтобы не нагружать серверы hh
    time.sleep(1)

print('Вакансии собраны')

vacancies_ids = []
for i in range(len(js_objs)):
    vacancies_ids.append(js_objs[i]['id'])

key_skills = []

for vacancy_id in vacancies_ids:
    resp = requests.get('https://api.hh.ru/vacancies/' + vacancy_id)
    key_skills.append(resp.json()['key_skills'])

key_skills_id = [
    {"key_skills": [el['name'] for el in lst]} for lst in key_skills
]

print('Ключевые навыки собраны')


df = pd.DataFrame(js_objs)

df.pop('accept_temporary')
df['key_skills'] = key_skills_id
df.pop('premium')
df.pop('department')
df.pop('has_test')
df.pop('response_letter_required')
df.pop('type')
df.pop('sort_point_distance')
df.pop('address')
df.pop('published_at')
df.pop('created_at')
df.pop('archived')
df.pop('insider_interview')
df.pop('apply_alternate_url')
df.pop('url')
df.pop('relations')
df.pop('working_time_intervals')
df.pop('working_time_modes')
df.pop('working_days')
df.pop('response_url')
df.pop('contacts')

df.to_csv('{}_{}_{}_{}.csv'.format(name, q, experience, today))

Вакансии собраны
Ключевые навыки собраны


In [5]:
df.head()

,id,name,area,salary,adv_response_url,alternate_url,employer,snippet,schedule,key_skills
0,67753157,Data analyst,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': None, 'to': 206897, 'currency': 'RUR'...",https://api.hh.ru/vacancies/67753157/adv_respo...,https://hh.ru/vacancy/67753157,"{'id': '2460946', 'name': 'Самокат (ООО Умный ...",{'requirement': 'Опыт работы маркетинговым ана...,"{'id': 'remote', 'name': 'Удаленная работа'}",{'key_skills': []}
1,67800224,Аналитик данных,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,https://api.hh.ru/vacancies/67800224/adv_respo...,https://hh.ru/vacancy/67800224,"{'id': '1102601', 'name': 'Группа Самолет', 'u...","{'requirement': 'Высшее образование, математик...","{'id': 'remote', 'name': 'Удаленная работа'}","{'key_skills': ['MS PowerPoint', 'Atlassian Ji..."
2,66984947,Аналитик,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,https://api.hh.ru/vacancies/66984947/adv_respo...,https://hh.ru/vacancy/66984947,"{'id': '2733062', 'name': 'Лига Цифровой Эконо...",{'requirement': 'Умение описывать требования к...,"{'id': 'remote', 'name': 'Удаленная работа'}","{'key_skills': ['SOAP', 'UML', 'Atlassian Jira..."
3,68551275,Product Analyst,"{'id': '1002', 'name': 'Минск', 'url': 'https:...",None,https://api.hh.ru/vacancies/68551275/adv_respo...,https://hh.ru/vacancy/68551275,"{'id': '859391', 'name': 'AWEM', 'url': 'https...",{'requirement': 'Опыт работы с данными от 1 го...,"{'id': 'remote', 'name': 'Удаленная работа'}","{'key_skills': ['Python', 'SQL', 'Статистическ..."
4,45351631,System Analyst,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,https://api.hh.ru/vacancies/45351631/adv_respo...,https://hh.ru/vacancy/45351631,"{'id': '3114205', 'name': 'ИММО', 'url': 'http...",{'requirement': 'Аналитическое мышление. Высок...,"{'id': 'remote', 'name': 'Удаленная работа'}",{'key_skills': []}


In [ ]:
# dictionaries = requests.get('https://api.hh.ru/dictionaries/') 
# print(dictionaries.json())

{'vacancy_label': [{'id': 'with_address', 'name': 'Только с адресом'}, {'id': 'accept_handicapped', 'name': 'Только доступные для людей с инвалидностью'}, {'id': 'not_from_agency', 'name': 'Без вакансий агентств'}, {'id': 'accept_kids', 'name': 'Только доступные для соискателей от 14 лет'}], 'resume_access_type': [{'id': 'no_one', 'name': 'не видно никому'}, {'id': 'whitelist', 'name': 'видно выбранным компаниям'}, {'id': 'blacklist', 'name': 'скрыто от выбранных компаний'}, {'id': 'clients', 'name': 'видно всем компаниям, зарегистрированным на HeadHunter'}, {'id': 'everyone', 'name': 'видно всему интернету'}, {'id': 'direct', 'name': 'доступно только по прямой ссылке'}], 'vacancy_search_order': [{'id': 'publication_time', 'name': 'по дате'}, {'id': 'salary_desc', 'name': 'по убыванию дохода'}, {'id': 'salary_asc', 'name': 'по возрастанию дохода'}, {'id': 'relevance', 'name': 'по соответствию'}, {'id': 'distance', 'name': 'по удалённости'}], 'vacancy_search_fields': [{'id': 'name', 'na

In [ ]:
# specializations = requests.get('https://api.hh.ru/specializations/') 
# print(specializations.json())

[{'id': '1', 'name': 'Информационные технологии, интернет, телеком', 'specializations': [{'id': '1.395', 'name': 'Банковское ПО', 'laboring': False}, {'id': '1.400', 'name': 'Оптимизация сайта (SEO)', 'laboring': False}, {'id': '1.420', 'name': 'Администратор баз данных', 'laboring': False}, {'id': '1.474', 'name': 'Стартапы', 'laboring': False}, {'id': '1.475', 'name': 'Игровое ПО', 'laboring': False}, {'id': '1.536', 'name': 'CRM системы', 'laboring': False}, {'id': '1.744', 'name': 'Другое', 'laboring': False}, {'id': '1.3', 'name': 'CTO, CIO, Директор по IT', 'laboring': False}, {'id': '1.9', 'name': 'Web инженер', 'laboring': False}, {'id': '1.10', 'name': 'Web мастер', 'laboring': False}, {'id': '1.25', 'name': 'Аналитик', 'laboring': False}, {'id': '1.30', 'name': 'Арт-директор', 'laboring': False}, {'id': '1.50', 'name': 'Системы управления предприятием (ERP)', 'laboring': False}, {'id': '1.82', 'name': 'Инженер', 'laboring': False}, {'id': '1.89', 'name': 'Интернет', 'laboring

In [ ]:
# filename = '{}_{}_{}_{}.csv'.format(name, q, experience, today)

# with open(filename, 'w', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=js_objs[0].keys(), extrasaction='ignore')
#     writer.writeheader()
#     for item in js_objs:
#         writer.writerow(item)